Import all the required headers

In [1]:
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import numpy as np
import sys
sys.path.append("../lib")
from load import faces
from theano.tensor.nnet.conv import conv2d
from theano.tensor.signal.downsample import max_pool_2d

Using gpu device 0: GeForce GTX 980


Helper functions

In [2]:

srng = RandomStreams()

def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def init_biases(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def rectify(X):
    return T.maximum(X, 0.)

def softmax(X):
    e_x = T.exp(X - X.max(axis=1).dimshuffle(0, 'x'))
    return e_x / e_x.sum(axis=1).dimshuffle(0, 'x')

def logistic(X):
    s = 1 / (1 + T.exp(-X))
    return s   
    
def dropout(X, p=0.):
    if p > 0:
        retain_prob = 1 - p
        X = X * srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X = (X/retain_prob)
    return X

def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates


Model

In [3]:
def model(X, w, w2, w3, w4, p_drop_conv, p_drop_hidden):
    l1a = T.tanh(T.dot(X,w)+b)


    pyx = softmax(T.dot(l4, w_o))
    return l1, l2, l3, l4, pyx


In [4]:
def autoEncoder(X,w):
    return X * w
    

Load Datasets into memory

In [ ]:
trX, teX, trY, teY = faces()
#trX = trX.reshape(-1, 1, 48, 48)
#teX = teX.reshape(-1, 1, 48, 48)
print trX.shape
print teX.shape


Do the Network thing

In [6]:
#initialize All the parameters
X = T.ftensor4()
Y = T.fmatrix()
w = init_weights(( 1, 48, 48))
b = init_biases(1,48)
w2 = init_weights((1, 24, 24))
w3 = init_weights((1, 12, 12))
w4 = init_weights((128 * 5 * 5, 625))
w_o = init_weights((625, 7))

In [7]:

#Train Loop
noise_l1, noise_l2, noise_l3, noise_l4, noise_py_x = model(X, w, w2, w3, w4, 0.2, 0.5)
cost = T.mean(T.nnet.categorical_crossentropy(noise_py_x, Y))
params = [w, w2, w3, w4, w_o]
updates = RMSprop(cost, params, lr=0.001)
train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)



In [8]:
#Predict Loop
l1, l2, l3, l4, py_x = model(X, w, w2, w3, w4, 0., 0.)
y_x = T.argmax(py_x, axis=1)
predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True)

Train the network

In [10]:
for i in range(287):
    for start, end in zip(range(0, len(trX), 100), range(100, len(trX), 100)):
        cost = train(trX[start:end], trY[start:end])
    print np.mean(np.argmax(teY, axis=1) == predict(teX))
    print "Interation number: " + str(i)



0.261354137643
Interation number: 0
0.425745332962
Interation number: 1
0.45528002229
Interation number: 2
0.485371969908
Interation number: 3
0.515185288381
Interation number: 4
0.512398996935
Interation number: 5
0.522429646141
Interation number: 6
0.54416271942
Interation number: 7
0.553636110337
Interation number: 8
0.554471997771
Interation number: 9
0.561994984675
Interation number: 10
0.563388130398
Interation number: 11
0.566731680134
Interation number: 12
0.567567567568
Interation number: 13
0.573418779604
Interation number: 14
0.575369183617
Interation number: 15
0.57620507105
Interation number: 16
0.579269991641
Interation number: 17
0.570632488158
Interation number: 18
0.572025633881
Interation number: 19
0.579269991641
Interation number: 20
0.582613541376
Interation number: 21
0.586792978546
Interation number: 22
0.585399832823
Interation number: 23
0.582056283087
Interation number: 24
0.581220395653
Interation number: 25
0.576483700195
Interation number: 26
0.56951797158
